### Converting video to images

In [2]:
import os
import cv2

image_path = 'images'
raw_video_path = 'raw_vid'
moods = os.listdir(image_path)


def video_conv(video_files, save_folder):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    j = 1 #counter of saved images
    
    for file in video_files:
        print("Extracting",file)
        video = cv2.VideoCapture(file)
        i = 0
        while True:
            i += 1
            ret, frame = video.read()
            if i!=2: continue
            i = 0
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            faces = face_cascade.detectMultiScale(gray, 1.5, 5)

            for (x, y, w, h) in faces:
                roi_gray = frame[y:y + h, x:x + w]
                roi = cv2.resize(roi_gray, (224,224), interpolation=cv2.INTER_LINEAR)
                cv2.imwrite(os.image_path.join(save_folder, f'image_{j}.jpg'), roi)
                j += 1
        video.release()

for mood in moods:
  print(mood)
  files = os.listdir(f'{raw_video_path}/{mood}')
  video_files = [os.image_path.join(raw_video_path,mood,file) for file in files if file.endswith(('.mp4', '.avi', '.MOV'))]
  video_conv(video_files,f'{image_path}/{mood}')

Angry
Extracting raw_vid\Angry\angry.MOV
Confused
Extracting raw_vid\Confused\confused.MOV
Happy
Extracting raw_vid\Happy\happy.MOV
Sad
Extracting raw_vid\Sad\sad.MOV


### Data augmentation

In [3]:
from keras.preprocessing.image import ImageDataGenerator

idg = ImageDataGenerator(rescale=1./255,
                         validation_split= 0.2,
                         )

train_gen = idg.flow_from_directory('images', batch_size=32, target_size=(224,224), class_mode='categorical', subset='training')
test_gen = idg.flow_from_directory('images', batch_size=32, target_size=(224,224), class_mode='categorical', subset='validation')

Found 908 images belonging to 4 classes.
Found 225 images belonging to 4 classes.


### Building model

In [4]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D

model_1 = Sequential()

model_1.add(Conv2D(32, (2,2), strides=1, input_shape = (224,224,3)))
model_1.add(Activation('relu'))
model_1.add(Conv2D(32, (2,2), strides=1))
model_1.add(Activation('relu'))

model_1.add(MaxPooling2D(2,2))
model_1.add(Dropout(0.25))

model_1.add(Conv2D(32, (2,2), strides=1))
model_1.add(Activation('relu'))
model_1.add(Conv2D(32, (2,2), strides=1))
model_1.add(Activation('relu'))

model_1.add(MaxPooling2D(2,2))
model_1.add(Dropout(0.25))

model_1.add(Conv2D(64, (2,2), strides=1))
model_1.add(Activation('relu'))
model_1.add(Conv2D(64, (2,2), strides=1))
model_1.add(Activation('relu'))

model_1.add(MaxPooling2D(2,2))
model_1.add(Dropout(0.25))

model_1.add(Conv2D(128, (2,2), strides=1))
model_1.add(Activation('relu'))
model_1.add(Conv2D(128, (2,2), strides=1))
model_1.add(Activation('relu'))

model_1.add(MaxPooling2D(2,2))
model_1.add(Dropout(0.25))

model_1.add(Flatten())
model_1.add(Dense(128, activation = 'relu'))
model_1.add(Dropout(0.25))
model_1.add(Dense(4, activation = 'softmax'))

model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 223, 223, 32)      416       
                                                                 
 activation (Activation)     (None, 223, 223, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      4128      
                                                                 
 activation_1 (Activation)   (None, 222, 222, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                        

### Training model

In [5]:
model_1.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics='accuracy')
model_1.fit(train_gen, validation_data=test_gen, epochs=25)

Epoch 1/25
29/29 [==============================] - 16s 296ms/step - loss: 1.3795 - accuracy: 0.2808 - val_loss: 1.3762 - val_accuracy: 0.2578
Epoch 2/25
29/29 [==============================] - 4s 137ms/step - loss: 1.3180 - accuracy: 0.3811 - val_loss: 1.2918 - val_accuracy: 0.6089
Epoch 3/25
29/29 [==============================] - 4s 142ms/step - loss: 1.0305 - accuracy: 0.5793 - val_loss: 0.9315 - val_accuracy: 0.7778
Epoch 4/25
29/29 [==============================] - 4s 138ms/step - loss: 0.7830 - accuracy: 0.7081 - val_loss: 0.6616 - val_accuracy: 0.9689
Epoch 5/25
29/29 [==============================] - 4s 140ms/step - loss: 0.5226 - accuracy: 0.8172 - val_loss: 0.3520 - val_accuracy: 0.9911
Epoch 6/25
29/29 [==============================] - 4s 142ms/step - loss: 0.3147 - accuracy: 0.8855 - val_loss: 0.2612 - val_accuracy: 0.9778
Epoch 7/25
29/29 [==============================] - 4s 139ms/step - loss: 0.1680 - accuracy: 0.9493 - val_loss: 0.1097 - val_accuracy: 0.9778
Epoch

In [8]:
model_1.save('mood_model.h5')

### Testing model

In [13]:
import cv2
import numpy as np

mood = ['Confused', 'Confused', 'Confused', 'Confused']
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = cascade.detectMultiScale(gray, 1.3, 3)
    
    for (x,y,w,h) in roi:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224,224))
        face = np.expand_dims(face, axis=0)
        
        pred = model_1.predict(face)
        label = np.argmax(pred)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, mood[label], (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 30ms/step
